In [1]:
!wget https://raw.githubusercontent.com/ArmanArabi/KantaiBert/main/kant.txt

--2024-06-16 14:30:39--  https://raw.githubusercontent.com/ArmanArabi/KantaiBert/main/kant.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11258065 (11M) [text/plain]
Saving to: ‘kant.txt.1’

kant.txt.1          100%[===================>]  10.74M  --.-KB/s    in 0.03s   

2024-06-16 14:30:40 (373 MB/s) - ‘kant.txt.1’ saved [11258065/11258065]



In [2]:
!curl -L https://raw.githubusercontent.com/ArmanArabi/KantaiBert/main/kant.txt

Streaming output truncated to the last 5000 lines.
relation (as that which can exist only as the consequence of something
else), and so it is really an identical proposition: “That which can
exist only as a consequence, has a cause.” In fact, when we have to
give examples of contingent existence, we always refer to changes, and
not merely to the possibility of conceiving the opposite.[37] But
change is an event, which, as such, is possible only through a cause,
and considered per se its non-existence is therefore possible, and we
become cognizant of its contingency from the fact that it can exist
only as the effect of a cause. Hence, if a thing is assumed to be
contingent, it is an analytical proposition to say, it has a cause.

 [37] We can easily conceive the non-existence of matter; but the
 ancients did not thence infer its contingency. But even the
 alternation of the existence and non-existence of a given state in a
 thing, in which all change consists, by no means proves the
 co

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Step 2:Installing Hugging Face Transformers

In [3]:
## We won't need TensorFlow here
!pip uninstall -y tensorflow

## Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-h5fxuub1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-h5fxuub1
  Resolved https://github.com/huggingface/transformers to commit eed9ed679878ada2f6d2eefccdbda368cabc88b1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
tokenizers                       0.19.1
transformers                     4.42.0.dev0


## Training a Tokenizer

In [4]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]
print(paths)
## Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>"])

['kant.txt', 'KantaiBERT/merges.txt']


## Step 4: Saving the files to disk

In [5]:
import os
token_dir = '/content/KantaiBERT'

if not os.path.exists(token_dir):
  os.makedirs(token_dir)
tokenizer.save_model('KantaiBERT')

['KantaiBERT/vocab.json', 'KantaiBERT/merges.txt']

##Step 5 Loading the Trained

In [6]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer("./KantaiBERT/vocab.json","./KantaiBERT/merges.txt")


In [7]:
print(tokenizer.encode("The Critique of Pure Reason."))

print(tokenizer.encode("The Critique of Pure Reason.").tokens)
print(tokenizer.encode("The Critique of Pure Reason.").offsets)
print(tokenizer.encode("The Critique of Pure Reason.").attention_mask)
print(tokenizer.encode("The Critique of Pure Reason.").special_tokens_mask)

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.']
[(0, 3), (3, 12), (12, 15), (15, 20), (20, 27), (27, 28)]
[1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0]


In [8]:
## BertProcessing has 2 arguman :
## 1- sep (Tuple[str, int]) == start
## 2- cls (Tuple[str, int]) == end
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
print(tokenizer._tokenizer.post_processor)
tokenizer.enable_truncation(max_length=512)


## take an example
print(tokenizer.encode("The Critique of Pure Reason."))
print('//////')
print(tokenizer.encode("The Critique of Pure Reason.").tokens)

Encoding(num_tokens=8, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
//////
['<s>', 'The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.', '</s>']


## step 6 : check_gpu

In [9]:
!nvidia-smi

Sun Jun 16 14:31:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
import torch
torch.cuda.is_available()

True

## Step 7: Defining the configuration of the Model

In [11]:
from transformers import RobertaConfig

In [12]:
Config = RobertaConfig(
    vocab_size=52000,
    max_position_embeddings=514,
    num_hidden_layers=6,
    type_vocab_size=1)

In [13]:
print(Config)

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.42.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}



## Step 8: Re-creating the Tokenizer in Transformers

In [14]:
from transformers import RobertaTokenizer

In [15]:
tokenizer = RobertaTokenizer.from_pretrained(
    '/content/KantaiBERT',
    max_length=512)

## Step 9: Initializing a Model From Scratch

In [16]:
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=Config)

In [17]:
print(model)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [18]:
## chaeck parameters

for i in model.parameters():
    print(i)
    print('\n')


Parameter containing:
tensor([[-0.0116,  0.0383,  0.0121,  ...,  0.0116,  0.0412, -0.0354],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0228, -0.0165, -0.0010,  ..., -0.0033, -0.0091,  0.0165],
        ...,
        [-0.0138,  0.0065, -0.0041,  ..., -0.0184, -0.0382, -0.0001],
        [-0.0064,  0.0178,  0.0078,  ..., -0.0136,  0.0349,  0.0116],
        [ 0.0161,  0.0267, -0.0133,  ..., -0.0129, -0.0142, -0.0099]],
       requires_grad=True)


Parameter containing:
tensor([[ 0.0136,  0.0002,  0.0129,  ...,  0.0160,  0.0067,  0.0019],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0086,  0.0176,  0.0081,  ...,  0.0226,  0.0050, -0.0098],
        ...,
        [-0.0115, -0.0032, -0.0160,  ...,  0.0170, -0.0041,  0.0160],
        [ 0.0005, -0.0285, -0.0056,  ..., -0.0048, -0.0361, -0.0177],
        [-0.0147,  0.0192,  0.0244,  ...,  0.0152, -0.0136, -0.0072]],
       requires_grad=True)


Parameter containing:
tensor([

In [19]:
## number of above metrics and vectors are :

LIST_PARAMETERS = list(model.parameters())
print(len(LIST_PARAMETERS))

106


In [20]:
## number of parameters
print(model.num_parameters())

83504416


In [21]:
## check the number of parameters for each param

number_of_parameters = 0

for param in range(0,len(LIST_PARAMETERS)):#number of tensors
    PL2=True
    try:
        #if len(LIST_PARAMETERS[0].shape) ==2
        L2=len(LIST_PARAMETERS[param][0]) #check if 2D
    except:
        L2=1 #not 2D but 1D
        PL2=False

    L1=len(LIST_PARAMETERS[param])
    L3=L1*L2
    number_of_parameters+=L3 # number of parameters per tensor
    if PL2==True:
        print(f'param:{param} -- L1:{L1} -- L2:{L2} -- L3:{L3}') # displaying the sizes of the parameters
    if PL2==False:
        print(f'param:{param} - -L1:{L1} -- L3:{L3}') # displaying the sizes of the parameters

print(f"number_of_parameters IS : {number_of_parameters}") # total number of parameters

param:0 -- L1:52000 -- L2:768 -- L3:39936000
param:1 -- L1:514 -- L2:768 -- L3:394752
param:2 -- L1:1 -- L2:768 -- L3:768
param:3 - -L1:768 -- L3:768
param:4 - -L1:768 -- L3:768
param:5 -- L1:768 -- L2:768 -- L3:589824
param:6 - -L1:768 -- L3:768
param:7 -- L1:768 -- L2:768 -- L3:589824
param:8 - -L1:768 -- L3:768
param:9 -- L1:768 -- L2:768 -- L3:589824
param:10 - -L1:768 -- L3:768
param:11 -- L1:768 -- L2:768 -- L3:589824
param:12 - -L1:768 -- L3:768
param:13 - -L1:768 -- L3:768
param:14 - -L1:768 -- L3:768
param:15 -- L1:3072 -- L2:768 -- L3:2359296
param:16 - -L1:3072 -- L3:3072
param:17 -- L1:768 -- L2:3072 -- L3:2359296
param:18 - -L1:768 -- L3:768
param:19 - -L1:768 -- L3:768
param:20 - -L1:768 -- L3:768
param:21 -- L1:768 -- L2:768 -- L3:589824
param:22 - -L1:768 -- L3:768
param:23 -- L1:768 -- L2:768 -- L3:589824
param:24 - -L1:768 -- L3:768
param:25 -- L1:768 -- L2:768 -- L3:589824
param:26 - -L1:768 -- L3:768
param:27 -- L1:768 -- L2:768 -- L3:589824
param:28 - -L1:768 -- L3

## Step 10: Building the Dataset with batchsize

In [22]:
from transformers import LineByLineTextDataset

In [23]:
## LineByLineTextDataset is useful to read the txt dataset, conversion and like these.
%%time
dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path="/content/kant.txt",
    block_size=128
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 19.9 s, sys: 321 ms, total: 20.3 s
Wall time: 20.3 s


##Step 11: Defining a Data Collator

In [24]:
from transformers import DataCollatorForLanguageModeling

In [25]:
## is useful more for MLM task to manage the nasks
## usually use with object data initilizer like LineByLineTextDataset
data_collector = DataCollatorForLanguageModeling(
    tokenizer,
    mlm=True,
    mlm_probability=0.15)

## Step 12: Initializing the Trainer

In [26]:
from transformers import Trainer , TrainingArguments

In [27]:
## check the version of accelerate and transformers for Trained modual
!pip show accelerate
!pip show transformers

Name: accelerate
Version: 0.31.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
Name: transformers
Version: 4.42.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [28]:
## install the last update of these packeages
!pip install --upgrade pip
!pip install accelerate --upgrade
!pip install transformers[torch] --upgrade

In [29]:
from accelerate import Accelerator

In [30]:
## Define train-arguments and use it for Trainer
training_args = TrainingArguments(
    output_dir="/content/KantaiBERT" ,
    overwrite_output_dir=True ,
    num_train_epochs=1 ,
    per_device_train_batch_size=64 , #each device(cpu or gpu) will process 64 example at a time
    save_steps=10000 , # savemodel and output evry 10000 steps
    save_total_limit=2) # save only 2 last checkpoint and delet older. help to manage disk

trainer = Trainer(
    model= model ,
    args= training_args ,
    data_collator= data_collector ,
    train_dataset= dataset
)



## Step 13: Pre-training the Model

In [31]:
%%time
trainer.train()
## the result include :
## 1-training process in real time,  2-loss,  3-learning_rate,  4-epochs,  5-steps

Step,Training Loss
500,6.601700
1000,5.749800
1500,5.280300
2000,5.044500
2500,4.930400


CPU times: user 9min 3s, sys: 18.7 s, total: 9min 21s
Wall time: 9min 34s


TrainOutput(global_step=2672, training_loss=5.478983485056255, metrics={'train_runtime': 574.6032, 'train_samples_per_second': 297.534, 'train_steps_per_second': 4.65, 'total_flos': 873691623267840.0, 'train_loss': 5.478983485056255, 'epoch': 1.0})

## saving the final model

In [32]:
trainer.save_model("/content/KantaiBERT")

## Step 15: Language Modeling with the FillMaskPipeline

In [33]:
from transformers import pipeline

In [39]:
fill_mask = pipeline(
    task='fill-mask',
    model="/content/KantaiBERT",
    tokenizer="/content/KantaiBERT"
)

In [40]:
fill_mask("Human thinking involves human <mask>.")


[{'score': 0.043949976563453674,
  'token': 394,
  'token_str': ' reason',
  'sequence': 'Human thinking involves human reason.'},
 {'score': 0.01425712089985609,
  'token': 419,
  'token_str': ' conception',
  'sequence': 'Human thinking involves human conception.'},
 {'score': 0.011628099717199802,
  'token': 610,
  'token_str': ' conceptions',
  'sequence': 'Human thinking involves human conceptions.'},
 {'score': 0.010866457596421242,
  'token': 674,
  'token_str': ' principles',
  'sequence': 'Human thinking involves human principles.'},
 {'score': 0.010839536786079407,
  'token': 396,
  'token_str': ' object',
  'sequence': 'Human thinking involves human object.'}]

In [41]:
fill_mask("Arman want to know about <mask> of the world.")

[{'score': 0.01447000540792942,
  'token': 689,
  'token_str': ' unity',
  'sequence': 'Arman want to know about unity of the world.'},
 {'score': 0.012826580554246902,
  'token': 673,
  'token_str': ' means',
  'sequence': 'Arman want to know about means of the world.'},
 {'score': 0.010333524085581303,
  'token': 394,
  'token_str': ' reason',
  'sequence': 'Arman want to know about reason of the world.'},
 {'score': 0.009996837005019188,
  'token': 588,
  'token_str': ' nature',
  'sequence': 'Arman want to know about nature of the world.'},
 {'score': 0.009816091507673264,
  'token': 610,
  'token_str': ' conceptions',
  'sequence': 'Arman want to know about conceptions of the world.'}]